In [ ]:
from wizard.atoms import SymbolInfo
from wizard.calculator import MaterialCalculator
from calorine.calculators import CPUNEP
from itertools import combinations_with_replacement
import numpy as np
from math import gcd
from tqdm import tqdm

hmax = 3

# set up hkls
hkls_cubic = set()
for hkl in combinations_with_replacement(range(hmax + 1), 3):
    hkl = tuple(np.array(np.array(hkl) / gcd(*hkl), dtype=int))
    if sum(hkl) <= 0:
        continue
    hkls_cubic.add(hkl)

hkls_hexagonal = set()
for l in range(0, hmax+1):
    for h, k in combinations_with_replacement(range(hmax + 1), 2):
        hkl = (h, k, l)
        hkl = tuple(np.array(np.array(hkl) / gcd(*hkl), dtype=int))
        if sum(hkl) <= 0:
            continue
        hkls_hexagonal.add(hkl)

symbol_infos = [
SymbolInfo('Ag', 'fcc', 4.146),
SymbolInfo('Al', 'fcc', 4.042),
SymbolInfo('Au', 'fcc', 4.159),
SymbolInfo('Cu', 'fcc', 3.631),
SymbolInfo('Ni', 'fcc', 3.509),
SymbolInfo('Pb', 'fcc', 5.038),
SymbolInfo('Pd', 'fcc', 3.939),
SymbolInfo('Pt', 'fcc', 3.967),
SymbolInfo('Cr', 'bcc', 2.845),
SymbolInfo('Mo', 'bcc', 3.164),
SymbolInfo('Ta', 'bcc', 3.319),
SymbolInfo('V',  'bcc', 2.997),
SymbolInfo('W',  'bcc', 3.185),
SymbolInfo('Mg', 'hcp', 3.195, 5.186),
SymbolInfo('Ti', 'hcp', 2.931, 4.651),
SymbolInfo('Zr', 'hcp', 3.240, 5.157)
]

for symbol_info in symbol_infos:
    atoms = symbol_info.create_bulk_atoms((1,1,1))
    calc = CPUNEP('../Train/continue_train/nep.txt')
    material_calculator = MaterialCalculator(atoms, calc, symbol_info)
    if symbol_info.structure == 'hcp':
        hkls = hkls_hexagonal
    else:
        hkls = hkls_cubic
    for hkl in tqdm(sorted(hkls)):
        material_calculator.formation_energy_surface(hkl) 

In [ ]:
from wizard.atoms import SymbolInfo
from wizard.calculator import MaterialCalculator
from calorine.calculators import CPUNEP

symbol_infos = [
SymbolInfo('Ag', 'fcc', 4.146),
SymbolInfo('Al', 'fcc', 4.042),
SymbolInfo('Au', 'fcc', 4.159),
SymbolInfo('Cu', 'fcc', 3.631),
SymbolInfo('Ni', 'fcc', 3.509),
SymbolInfo('Pb', 'fcc', 5.038),
SymbolInfo('Pd', 'fcc', 3.939),
SymbolInfo('Pt', 'fcc', 3.967),
SymbolInfo('Cr', 'bcc', 2.845),
SymbolInfo('Mo', 'bcc', 3.164),
SymbolInfo('Ta', 'bcc', 3.319),
SymbolInfo('V',  'bcc', 2.997),
SymbolInfo('W',  'bcc', 3.185),
SymbolInfo('Mg', 'hcp', 3.195, 5.186),
SymbolInfo('Ti', 'hcp', 2.931, 4.651),
SymbolInfo('Zr', 'hcp', 3.240, 5.157)
]

for symbol_info in symbol_infos:
    atoms = symbol_info.create_bulk_atoms((1,1,1))
    calc = CPUNEP('../Train/continue_train/nep.txt')
    material_calculator = MaterialCalculator(atoms, calc, symbol_info)
    e, _ = material_calculator.lattice_constant()
    with open('energy.txt', 'a') as f:
        f.write(f'{symbol_info.formula} {e}\n')

In [1]:
element_energy = {
    "Ag": -2.7186902407477165,
    "Al": -3.7399009053578087,
    "Au": -3.224828187747349,
    "Cu": -3.731157012534359,
    "Ni": -5.413197564413901,
    "Pb": -3.5593040153055817,
    "Pd": -5.22007624214573,
    "Pt": -6.1056948416596875,
    "Cr": -9.511080124865572,
    "Mo": -10.937464162256912,
    "Ta": -11.816886272670867,
    "V": -8.998021822480961,
    "W": -12.953523326685003,
    "Mg": -1.504670525267171,
    "Ti": -7.829707702236345,
    "Zr": -8.521373915892298
}

In [4]:
from calorine.calculators import CPUNEP
from ase.io import read
from ase.units import J
import os

def find_cif_files(directory):
    cif_files = []
    for root, dirs, files in os.walk(directory):
        if 'FS' in root.split(os.sep):
            for file in files:
                if file.endswith('.cif'):
                    cif_files.append(os.path.join(root, file))
    return cif_files

calc = CPUNEP('../Train/continue_train/nep.txt')
directory = '../Generate/GB-FS'  
cif_files = find_cif_files(directory)
for file in cif_files:
    atoms = read(file)
    atoms.calc = calc
    box = atoms.get_cell()
    S = (box[0][0] * box[1][1] - box[0][1] * box[1][0])
    e = (atoms.get_potential_energy() - element_energy[atoms.get_chemical_symbols()[0]] * len(atoms)) / S / 2 / J / 1e-20
    
    name = os.path.splitext(os.path.basename(file))[0]
    with open('surface_energy', 'a') as f:
        f.write(f'{name} {e} \n')
